![data-x](https://raw.githubusercontent.com/afo/data-x-plaksha/master/imgsource/dx_logo.png)


# Data-X Notebook: NLP & NLTK
#### Using Natural Language Processing in Python to do sentiment analysis on IMDB movie reviews

Author: Alexander Fred Ojala

Source: 
- https://www.kaggle.com/c/word2vec-nlp-tutorial/data
- https://github.com/rasbt/python-machine-learning-book/tree/master/code
- https://github.com/justmarkham/pycon-2016-tutorial

*Code snippets given run on both Python 2.7 and Python 3.x


# Table of Contents:

### - [Part 0: Pre-Setup & Data description](#sec0)
### - [Part 1: Explore the Data](#sec1)
### - [Part 2: NLTK intro (1 review) cleaning, tokenizing, stemming, lemmatization, stopwords](#sec2)
### - [Part 3: Preparing the data set for classification](#sec3)
### - [Part 4: Sentiment Classification w scikit-learn, Feature vectors & Bag of Words model](#sec4)
#### - [Appendix: Submitting to Kaggle & possible extensions](#sec5)

<div id='sec0'></div>
## Part 0: Pre-Setup & Data description

In [ ]:
# Remove warnings
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

#make compatible with Python 2 and Python 3
from __future__ import print_function, division, absolute_import 

# Data description

You can download the data (labeledTrainData.tsv.zip) here: https://www.kaggle.com/c/word2vec-nlp-tutorial/data, place it in your working directory & unzip the file. (It is placed in the data folder if you cloned the Github repo)

## Data set

The labeled training data set consists of 25,000 IMDB movie reviews. There is also an unlabeled test set with 25,000 IMDB movie reviews. The sentiment of the reviews are binary, meaning an IMDB rating < 5 results in a sentiment score of 0, and a rating >=7 have a sentiment score of 1 (no reviews with score 5 or 6 are included in the analysis). No individual movie has more than 30 reviews.

## File description

* **labeledTrainData** - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review. 

* **testData** - The unlabeled test set. 25,000 rows containing an id, and text for each review. 

## Data columns
* **id** - Unique ID of each review
* **sentiment** - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* **review** - Text of the review


# Part 1: Explore the data

In [ ]:
!head -n4 data/labeledTrainData.tsv 
# use bash command to see data set structure

In [ ]:
# Read in the data to a Pandas data frame
# Use header = 0 (first line contains col names)
# use delimiter=\t (columns are separated by tabs),
# use quoting=3 (Python will ignore doubled quotes)

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd       
train = pd.read_csv("data/labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)
# train.shape should be (25000,3)


test = pd.read_csv("data/testData.tsv", header=0, \
                    delimiter="\t", quoting=3)

# Data set statistics


In [ ]:
train.head()

In [ ]:
train.info() # no NaN's

### 1: How many movie reviews are positive and how many are negative in labeledTrainData.tsv? Do we have balance between the classes?

In [ ]:
# 1. 
print(train.shape)
print()
print('Number of Data Samples for every label output. 1=postive, 2=negative:')
print(train.sentiment.value_counts())

train.sentiment.hist(); #class balance

### 2: What is the average length of all the reviews (string length)?

In [ ]:
# 2. Apply length function to the review column
lengths = train.review.apply(len)

print('Average character length of the reviews are:')
print (np.mean(lengths))

<div id='sec2'></div>
# Part 2: 
## NLTK intro (1 review) cleaning, tokenizing, stemming, lemmatization, stopwords

Let's explore NLP by looking at the third review in the training data set, i.e. `train['review'][2]`

In [ ]:
# import packages

import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

eng_stopwords = stopwords.words('english')

In [ ]:
review3 = train['review'][2] # the review used for initial analysis
print(review3)

## Clean the review

First we would like to clean up the reviews. As you can see many interviews contain \ characters in front of quotation symobols, "`<br/>` tags, numbers, abbrevations etc.

### 1: Remove all the HTML tags in the third review, by creating a beatifulsoup object and then using the `.text` method. Save results in variable `review3

In [ ]:
# 1.
review3 = bs.BeautifulSoup(review3,features='lxml').text # removes HTML tags
print(review3[:1150])

### 2: Import NLTK's sent_tokenizer and count the number of sentences in review 3 
The review should be cleaned from HTML tags. To import sent_tokenizer use: `from nltk.tokenize import sent_tokenize`

In [ ]:
print(len(sent_tokenize(review3)))
sent_tokenize(review3) # doesn't really split all sentences

In [ ]:
# Check if it does a better job if we add space after every period
review3 = review3.replace('.','. ')

print(len(sent_tokenize(review3)), end='\n\n') # number of sentences

# print all sentences on a new line
for sent in sent_tokenize(review3):
    print(sent, end='\n\n')

### 3: Remove all punctuation and special characters from the third review.
We can do this using Regular Expression - package `re`. Regular expression define search patterns for text that can be used to search, replace, substitute etc. certain patterns in text data.

We'll use regular expression to only look for alphabetical characters, `[^a-zA-Z]`. Save results in variable `review3`

Intro to Regex: http://www.aivosto.com/vbtips/regex.html

In [ ]:
review3 = re.sub('[^a-zA-Z]',' ',review3)
print(review3) # remove special characters

### 4: Convert all the letters to lower case, split the string so that every word is one element in a list
Note: When we split the strings into words that process is called tokenization.

In [ ]:
review3 = review3.lower()
review3

In [ ]:
review3_words = review3.split()
print(review3_words[:10]) # tokenize and lower case

## 5: Stemming
Use NLTK's PorterStemmer (`from nltk.stem import PorterStemmer`). Create a new Porter stemmer (`stemmer = PorterStemmer()`) and run it on every word in `review3_words`, print the results as one string (don't overwrite the `review3_words` variable from 4). 

**What does the PorterStemmer do?**
PorterStemmer tries treating similar words as the same (e.g. give, gives, given = give).

**Stemming (paraphrased from Wiki):** Stemming reduces words to their word stem, base or root form (i.e. similar words, with the same stem, will be interpreted as the same)

In [ ]:

ps = PorterStemmer() #initialize Porter Stemmer object

ps_stems = []
for w in review3_words:
    ps_stems.append(ps.stem(w))

print(' '.join(ps_stems)) # add all the stemmed words to one string

### 6 Part of Speech tagging
Now we want to Part Of Speech (POS)) tag the third movie review. We will use POS labeling, also called grammatical tagging. To do this import `from nltk.tag import pos_tag`. 

When you use `pos_tag` on a word it returns a token-tag pair in the form of a tuple. In NLTK's Penn Treebank POS, the abbreviation (tag) for an adjective is JJ and NN for singular nouns. 

Let's count the number of singular nouns (NN) and adjectives (JJ) in `review3_words` using NLTK's pos_tag. A list of the Penn Treebank pos_tag's can be found here: http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [ ]:
# 6.
token_tag = pos_tag(review3_words)
token_tag[:10]

In [ ]:
# Cell that count nouns (NN) and adjectives (JJ) in review3
NN_count = 0
JJ_count = 0

for pair in token_tag:
    tag = pair[1]
    if tag == 'JJ':
        JJ_count+=1
    elif tag == 'NN':
        NN_count+=1
print('Number of nouns:', NN_count)
print('Number of adjectives:', JJ_count)

## 7. Lemmatizing
An even more sophisticated operation than stemming using the PorterStemmer is called lemmatizing. 

Lemmatizing, in contrast to stemming, does not create non-existent words and converts words to their synonyms. In order to use lemmatizing we need to define the wordnet POS tag. A function that takes in a POS Penn Treebank tag and converts it to a wordnet tag and then lemmatizes words in a string has been given written below. 

**Lemmatization (paraphrased from Wiki):** Lemmatization is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form.

We can use this to print the lemmatized third movie review.

In [ ]:
# 7.

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

In [ ]:
from nltk.stem import WordNetLemmatizer


wnl = WordNetLemmatizer()

wnl_stems = []
for pair in token_tag:
    res = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
    wnl_stems.append(res)

print(' '.join(wnl_stems))

## 8. StopWords

Lastly we will remove common words that don't carry much information. These are called stopwords. 

In English they could for example be 'am', 'are', 'and' etc. 

To import NLTK's list of stopwords you need to download the stopword corpora (`import nltk` and then `nltk.download()` if you don't have it). 

When that is done run `from nltk.corpus import stopwords` and create a variable for English stopwords with `eng_stopwords = stopwords.words('english')`. Use the list of English stopwords to remove all the stopwords from your list of words in the third movie review, i.e. `review3_words`. Print `review3_words` without stopwords, count the number of stopwords removed and print them as well.

In [ ]:
from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
print(eng_stopwords)

In [ ]:
review3_wo_stopwords = [w for w in review3_words if not w in stopwords.words("english")]
removed_stopwords = [w for w in review3_words if w in stopwords.words("english")]

print('REVIEW WITHOUT STOPWORDS:')
print(' '.join(review3_wo_stopwords))
print()
print('Stop words removed', removed_stopwords)
print()
print('NUMBER OF STOPWORDS REMOVED:',len(removed_stopwords))

<div id='sec3'></div>
# Part 3: Preparing the data set for classification

In this part we'll put everything that we've learned so far together. This so that we can use our reviews to make accurate predicitons on their sentiment.

## 1. Review cleaning function

We'll create a function called `review_cleaner` that reads in a review and:

- Removes HTML tags (using beautifulsoup)
- **Extract emoticons (emotion symbols, aka smileys :D )**
- Removes non-letters (using regular expression)
- Converts all words to lowercase letters and tokenizes them (using .split() method on the review strings, so that every word in the review is an element in a list)
- Removes all the English stopwords from the list of movie review words
- Join the words back into one string seperated by space, append the emoticons to the end

**NOTE: Transform the list of stopwords to a set before removing the stopwords. I.e. assign `eng_stopwords = set(stopwords.words("english"))`. Use the set to look up stopwords. This will speed up the computations A LOT (Python is much quicker when searching a set than a list).**

In [ ]:
# 1. 
from nltk.corpus import stopwords


def review_cleaner(review):
    '''
    Clean and preprocess a review.
    
    1. Remove HTML tags
    2. Use regex to remove all special characters (only keep letters)
    3. Make strings to lower case and tokenize / word split reviews
    4. Remove English stopwords
    5. Rejoin to one string
    '''
    
    #1. Remove HTML tags
    review = bs.BeautifulSoup(review).text
    
    #2. Use regex to find emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', review)
    
    #3. Remove punctuation
    review = re.sub("[^a-zA-Z]", " ",review)
    
    #4. Tokenize into words (all lower case)
    review = review.lower().split()
    
    #5. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))
    review = [w for w in review if not w in eng_stopwords]
    
    #6. Join the review to one sentence
    review = ' '.join(review+emoticons)
    # add emoticons to the end

    return(review)

## 2. Create original, Porter Stemmed, and Lemmatized data sets
    
**Create three lists `review_clean_original`, `review_clean_ps` and `review_clean_wnl`.**

- `review_clean_original` contains all the reviews from the train DataFrame, that have been cleaned by the function `review_cleaner` defined above.

- `review_clean_ps` applies the PorterStemmer to the reviews in `review_clean_original`. **Note:** NLTK version 3.2.2 crashes when trying to use the PorterStemming on the string 'oed' (known bug). Therefore, use an if statement to skip just that specific string/word.

- `review_clean_wnl` contains words that have been lemmatized using NLTK's WordNetLemmatizer on the words in the list `review_clean_original`.
    
**Note: This can take more than 10minutes to run on a laptop**

In [ ]:
%%time

num_reviews = len(train['review'])

review_clean_original = []

for i in range(0,num_reviews):
    if( (i+1)%500 == 0 ):
        # print progress
        print("Done with %d reviews" %(i+1)) 
    review_clean_original.append(review_cleaner(train['review'][i]))

In [ ]:
%%time
# Porter stemming on the results in review_clean_original

review_clean_ps = []

ps = PorterStemmer()


for i in range(0,num_reviews):
    if( (i+1)%500 == 0 ):
        # print progress
        print("Done with %d reviews" %(i+1)) 
    ps_stems = []
    for w in review_clean_original[i].split():
        if w == 'oed':
            continue
        ps_stems.append(ps.stem(w))
    
    review_clean_ps.append(' '.join(ps_stems))

In [ ]:
%%time
# Lemmatizer

review_clean_wnl = []

wnl = WordNetLemmatizer()

for i in range(0,num_reviews):
    if( (i+1)%500 == 0 ):
        # print progress
        print("Done with %d reviews" %(i+1)) 
    
    wnl_stems = []
    token_tag = pos_tag(review_clean_original[i].split())
    for pair in token_tag:
        res = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
        wnl_stems.append(res)

    review_clean_wnl.append(' '.join(wnl_stems))

<div id='sec4'></div>
# Part 4: Sentiment Classification w scikit-learn, Feature vectors & Bag of Words model

### Bag of Words: Explanation

Derived from source: https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words

We will now use scikit-learn to create numeric representations of the words in the reviews, using a method called Bag of Words. You can see this as learning a vocabulary from all the reviews and counting how many times a word appears in the reviews. For example, if we have two sentences:

**Sentence 1:** "cool students study cool data science"

**Sentence 2:** "to know data science study data science"

The vocabulary of these two sentences can be summarized in a dictionary:

    { cool, students, study, data, science, to, know }

The bags of words count the number of times each word occur in a sentence. In Sentence 1, "cool" appears twice, and "students", "study", "data", and "science" appear once. The feature vector for Sentence 1 is:

    Sentence 1: { 2, 1, 1, 1, 1, 0, 0 }

    Sentence 2: { 0, 0, 1, 2, 2, 1, 1 }


The `fit` method in scikit-learns `CountVectorizer` creates the vocabulary of the bag-of-words model and fits the function to the data set you pass in.

The`transform` method in `CountVectorizer` transforms text input to a feature matrix, where the rows are the document inputs and the columns are the words / feature in the vocabulary.

In [ ]:
## Example code BoW

from sklearn.feature_extraction.text import CountVectorizer

sent1 = "cool students study cool data science"
sent2 = "to know data science study data science"

vect = CountVectorizer() #instantiate

sents = np.array([sent1,sent2])

vect.fit(sents)


In [ ]:
print('Total number of words in the vocabulary (and position in feature matrix):\n')
print(vect.vocabulary_)

# vocabulary for the BoW model is stored in a dictionary

In [ ]:
# Transform to get feature vectors

bag = vect.transform(sents)

bag.toarray()

# the rows corresponds to the sentences 

In [ ]:
vect.get_feature_names() # stored in the right places

In [ ]:
# Put it in a DataFrame for interpretability

pd.DataFrame(bag.toarray(), columns=vect.get_feature_names(), index=[sent1,sent2])

# the number in the DataFrame is called Raw Term frequency raw term frequencies: 
# tf (t,d)—the number of times a term t occurs in a document d.

# Applying this strategy to the IMDB movie reviews

The movie review data contains a lot of words. To limit the analysis we use the 5000 most frequent words from the cleaned reviews. To extract the bag of words features we will use scitkit-learn.

The training data will be created by the `CountVectorizer` function from scikit-learn, and the training array will have 25000 rows (one for each review) and 5000 features (one for each vocabulary word).

CountVectorizer can automatically handle text cleaning, but here we specify `None`, instead we did a step-by-step cleaning of the data in the earlier problems.

#### Random Forest for review sentiment classification

First split up the data set so that 80% are used as training samples (the first 20000 reviews and their sentiment) and 20% are used as validation samples (the last 5000 reviews and their sentiment). Use Random Forest to do numeric training on the features for the training samples from the Bag of Words and their respective sentiment labels for each review / feature vector. The number of trees is set to 50 as a default value.


## Classify IMDB movie review dataset

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics # for confusion matrix, accuracy score etc
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(\
    review_clean_original, train['sentiment'], random_state=0, test_size=.2)


# CountVectorizer can actucally handle a lot of the preprocessing for us
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

In [ ]:
%%time
# Transform the text data to feature
# Only fit training data (to mimic real world)

vectorizer.fit(X_train)

In [ ]:
# Check that it worked, 
# now we have fitted a model that can transform features
# to sparse matrix representation

print(vectorizer.get_feature_names()[:10])

In [ ]:
train_bag = vectorizer.transform(X_train) #transform to a feature matrix
test_bag = vectorizer.transform(X_test)

In [ ]:
print(train_bag.toarray().shape) # 20,000 reviews, 2,000 feartures. just as expected
print(test_bag.toarray().shape)

In [ ]:
type(train_bag) # sparse matrix representation

print(train_bag)

## Clasify with Random Forest model

* Fit a Random Forest model to our bagged data set in order to do the sentiment analysis on `review_clean_original` and print the **validation accuracy** by using `forest.predict(test_bag)` and then comparing the resulting sentiment predictions with the ones stored in `y_test`.

*This can take 2-3 mins to run*

In [ ]:
from sklearn.ensemble import RandomForestClassifier

## Initialize a Random Forest classifier with 50 trees
# hyperparameter n_estimators always set in instantiation

forest = RandomForestClassifier(n_estimators = 50) 

In [ ]:
%%time
# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the target variable

forest = forest.fit(train_bag, y_train) # can take 2 mins to run

In [ ]:
# Make predictions

train_predictions = forest.predict(train_bag)
valid_predictions = forest.predict(test_bag)

## Accuracy

In [ ]:
metrics.accuracy_score(y_train,train_predictions) # 100% training accuracy

In [ ]:
metrics.accuracy_score(y_test,valid_predictions) # 83% test

In [ ]:
# Confusion matrix
# Is the number of False Positives and True negatives approx 50/50?
metrics.confusion_matrix(y_test,valid_predictions)

In [ ]:
valid_predictions==1

In [ ]:
# What are the characteristics of False Positives for example?
# Good practice when doing analysis

df_test = pd.DataFrame(X_test)
df_test[(y_test.values==0) & (valid_predictions==1)]

## Feature importances

In [ ]:
importances = forest.feature_importances_
# returns relative importance of all features.
# they are in the order of the columns
print(importances)

In [ ]:
# sort importance scores
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
top_10 = indices[:10]

# Get top ten features
print([vec.get_feature_names()[ind] for ind in top_10])

# Put everything together in a function

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# put everything together in a function

def predict_sentiment(cleaned_reviews, y=train["sentiment"]):

    print("Creating the bag of words model!\n")
    # CountVectorizer" is scikit-learn's bag of words tool, here we show more keywords 
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 2000) 
    
    X_train, X_test, y_train, y_test = train_test_split(\
    cleaned_reviews, y, random_state=0, test_size=.2)

    # Then we use fit_transform() to fit the model / learn the vocabulary,
    # then transform the data into feature vectors.
    # The input should be a list of strings. .toarraty() converts to a numpy array
    
    train_bag = vectorizer.fit_transform(X_train).toarray()
    test_bag = vectorizer.transform(X_test).toarray()

    # You can extract the vocabulary created by CountVectorizer
    # by running print(vectorizer.get_feature_names())


    print("Training the random forest classifier!\n")
    # Initialize a Random Forest classifier with 75 trees
    forest = RandomForestClassifier(n_estimators = 50) 

    # Fit the forest to the training set, using the bag of words as 
    # features and the sentiment labels as the target variable
    forest = forest.fit(train_bag, y_train)


    train_predictions = forest.predict(train_bag)
    test_predictions = forest.predict(test_bag)
    
    train_acc = metrics.accuracy_score(y_train, train_predictions)
    valid_acc = metrics.accuracy_score(y_test, test_predictions)
    print("The training accuracy is: ", train_acc, "\n", "The validation accuracy is: ", valid_acc)
    print()
    print()
    #Extract feature importnace
    print('TOP TEN IMPORTANT FEATURES:')
    importances = forest.feature_importances_
    indices = np.argsort(importances)[::-1]
    top_10 = indices[:10]
    print([vec.get_feature_names()[ind] for ind in top_10])

## Compare Original cleaned to lemmatized and stemmed data set

Now carry out the same analysis as above but on the `review_clean_ps` and `review_clean_wnl`. 

What data preprocessing strategy worked the best? Why do you think that is? (Feel free to change the number of features extracted in the bag of words model and the number of trees in the random forest model (i.e. the hyperparameters in our model), to see how it effects your accuracy. Is the accuracy better or worse?

In [ ]:
%%time

print('Original Reviews')
forest1,vec1 = predict_sentiment(review_clean_original)
print('Porter Stemmer')
forest2,vec2 = predict_sentiment(review_clean_ps)
print('Lemmatizing')
forest3,vec3 = predict_sentiment(review_clean_wnl)


# It  seems like Porter Stemmer and Lemmatizing does not effect the results as much as we thought
# This is just what Sebastian Raschka points out in his book Python Machine Learning:

'''
The Porter stemming algorithm is probably the oldest and simplest
stemming algorithm. Other popular stemming algorithms include the
newer Snowball stemmer (Porter2 or "English" stemmer) or the Lancaster
stemmer (Paice-Husk stemmer), which is faster but also more aggressive
than the Porter stemmer. Those alternative stemming algorithms are also
available through the NLTK package (http://www.nltk.org/api/
nltk.stem.html).

While stemming can create non-real words, such as thu, (from thus) as
shown in the previous example, a technique called lemmatization aims to
obtain the canonical (grammatically correct) forms of individual words—
the so-called lemmas. However, lemmatization is computationally more
diffcult and expensive compared to stemming and, in practice, it has
been observed that stemming and lemmatization have little impact on the
performance of text classifcation (Michal Toman, Roman Tesar, and Karel
Jezek. Infuence of word normalization on text classifcation. Proceedings of
InSciT, pages 354–358, 2006).
'''

<div id='sec5'></div>
## Appendix Kaggle submission code

In [ ]:
# If you want to submit to Kaggle
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print(test.shape)

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_cleaner( test["review"][i] )
    clean_test_reviews.append( clean_review )



# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "BoW_results.csv", index=False, quoting=3 )

# Strategies to improve accuracy



# Extra Credit (worth 1p)

* **Question:** Preprocess the reviews in any way you find suitable and build your own ML model that can predict the sentiment of movie reviews. Credit will be given if you can obtain a prediction accuracy of over 90%, when predicting the sentiments of the validation set (the last 5000 reviews). Train your model on the first 20000 reviews (with their sentiment as the target variable).

In [ ]:
## See other solution notebook ##